## step 0: initialize blockchain with genesis block
a genesis block is the only Block that is added to the chain without proof of work.

In [ ]:
from PyBlockchain import Miner, Node, Block, Blockchain, util

In [ ]:
# create genesis block with four entries
genesis_block = Block(['tx0', 'tx1', 'tx2', 'tx3'])
util.show(genesis_block)

In [ ]:
my_blockchain = Blockchain(genesis_block)  # add genesis block to blockchain

## step 1: create a new block
nodes create a new Block for batches of entries. 

it's not enough to create a new Block. a Block needs to demonstrate proof of work - that is, a Block nonce must satisfy certain conditions - before it can be added to the blockchain.

In [ ]:
# create two nodes to record entries on blocks
Node_Alice = Node(my_blockchain)
Node_Bob = Node(my_blockchain)

In [ ]:
# alice node creates a new Block, which cannot be added until an acceptable nonce is found
block1 = Node_Alice.generate_new_block(['S2M', '23', '13', '13'])
util.show(block1)

## step 2: add new Block to the chain
to add a new Block to the blockchain, it needs an acceptable Block hash. 

a Block's hash is the hash of (1) a nonce $N$, where $N \in \mathbb{R}$; (2) previous Block's hash (already verified); and (3) current Block's root hash, or the merkle root of entries in the current Block.

for example, a condition might be: "sha256(nonce + previous Block's hash + current Block's root hash) must start with four zeros". since the previous Block's hash and the current Block's root hash are constants, the only way to satisfy this condition is to toggle the *nonce*. 

finally, by including the previous Block's hash, the current Block's hash effectively seals itself with its predecessor.

In [ ]:
# a Block's hash cannot be added to the blockchain until it has an acceptable nonce
Node_Alice.update_blockchain(block1)

## step 3: miners demonstrate proof of work
through brute force, miners identify an acceptable nonce (there might be more than one) that results in an acceptable block hash (one that satsifies a specific condition).

In [ ]:
miner_sfo = Miner()  # miners are nodes that search for an acceptable hash by guessing nonces

In [ ]:
good_hash = miner_sfo.calculate_hash_block(block1)
print ("valid hash:", good_hash)

In [ ]:
# miner updates Block with an acceptable nonce
miner_sfo.update_block_hash(block1, good_hash)
util.show(block1)

In [ ]:
# verified new Block can be added by the node
Node_Alice.update_blockchain(block1)

In [ ]:
util.show(block1)

In [ ]:
# lets show one more block
block2 = Node_Alice.generate_new_block(['M1J', 'T6H', 'S2X', 'G4B'])
util.show(block2)

In [ ]:
Node_Alice.check_block(block2)  # again, new Block cannot be added until Block's hash is acceptable 

In [ ]:
good_hash = miner_sfo.calculate_hash_block(block2)
print ("valid hash:", good_hash)

In [ ]:
miner_sfo.update_block_hash(block2, good_hash)

In [ ]:
Node_Alice.check_block(block2)  # verify new block can be added to blockchain

In [ ]:
util.show(block2)

In [ ]:
# with the new hash, verify new block can be added to blockchain
Node_Alice.update_blockchain(block2)

In [ ]:
# examine history
for blocks in Node_Alice.blockchain_copy.blockchain:
    print (blocks.index, blocks.ls_transactions)

In [ ]:
# see if record 2 is valid
print ("hash signatures match?", 
Node_Alice.blockchain_copy.blockchain[1].block_hash == miner_sfo.calculate_hash_block(Node_Alice.blockchain_copy.blockchain[1]))

### what if record 1 was altered?

In [ ]:
# examine history
for blocks in Node_Alice.blockchain_copy.blockchain:
    print (blocks.index, blocks.ls_transactions)

In [ ]:
Node_Alice.blockchain_copy.blockchain[1].ls_transactions = ['s32', 'f2c', 'd3c', 'g67']

In [ ]:
# examine history
for blocks in Node_Alice.blockchain_copy.blockchain:
    print (blocks.index, blocks.ls_transactions)

In [ ]:
# see if record 2 is valid
print ("hash signatures match?", 
Node_Alice.blockchain_copy.blockchain[1].block_hash == miner_sfo.calculate_hash_block(Node_Alice.blockchain_copy.blockchain[1]))